# Time Series Data 전처리
- Source:
    - https://github.com/ritvikmath/Time-Series-Analysis

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# 데이터 확보

In [ ]:
def parser(s):
    return datetime.strptime(s, '%Y-%m')

In [ ]:
ice_cream_heater_df = pd.read_csv('data/ice_cream_vs_heater.csv', parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

In [ ]:
ice_cream_heater_df = ice_cream_heater_df.asfreq(pd.infer_freq(ice_cream_heater_df.index))

In [ ]:
heater_series = ice_cream_heater_df.heater

In [ ]:
heater_series

In [ ]:
def plot_series(series):
    plt.figure(figsize=(12,6))
    plt.plot(heater_series, color='red')
    plt.ylabel('Search Frequency for "Heater"', fontsize=16)

    for year in range(2004, 2021):
        plt.axvline(datetime(year,1,1), linestyle='--', color='k', alpha=0.5)

In [ ]:
plot_series(heater_series)

## 데이터를 Stationary 하게 만들기 위해
- 평균 일정
- 표준편차 일정
- No 계절성

# Standardization(표준화)을 통해 값의 범위를 조정
 $z =\frac{x_i-\mu}{\sigma} $


In [ ]:
avg, dev = heater_series.mean(), heater_series.std()

In [ ]:
heater_series = (heater_series - avg) / dev

In [ ]:
plot_series(heater_series)
plt.axhline(0, linestyle='--', color='k', alpha=0.3)

# 트렌드 제거
- 차분을 통해
- 근접한 값들의 차를 구하게 되면, 우상향 그래프라고 하더라도, 결국에는 이전값과의 차이로만 구성된 그래프가 그려짐

In [ ]:
heater_series = heater_series.diff().dropna()

In [ ]:
plot_series(heater_series)
plt.axhline(0, linestyle='--', color='k', alpha=0.3)

# 변동성 제거
- 월별 표준편차를 구하고, 해당월을 표준편차로 나눠주는 식으로
- 표준편차가 큰(변동성이 큰) 월들은 더 많이 나눠질 것이고, 적은(변동성이 적은) 월들은 덜 나눠진다.
- Scale이 비슷해진다

In [ ]:
annual_volatility = heater_series.groupby(heater_series.index.year).std()

In [ ]:
annual_volatility

In [ ]:
heater_annual_vol = heater_series.index.map(lambda d: annual_volatility.loc[d.year])

In [ ]:
heater_annual_vol

In [ ]:
heater_series = heater_series / heater_annual_vol

In [ ]:
plot_series(heater_series)
plt.axhline(0, linestyle='--', color='k', alpha=0.3)

# 계절성 제거
- 월별 평균값을 구하고, 해당 월에서 평균값을 빼기
- 월별로 비슷한 값들은 모두 0에 가까워진다.
- 0에 가까워지지 않은 값들은 계절성에 위반되는 예외적인값들: 잔차

In [ ]:
month_avgs = heater_series.groupby(heater_series.index.month).mean()

In [ ]:
month_avgs

In [ ]:
heater_month_avg = heater_series.index.map(lambda d: month_avgs.loc[d.month])

In [ ]:
heater_month_avg

In [ ]:
heater_series = heater_series - heater_month_avg

In [ ]:
plot_series(heater_series)
plt.axhline(0, linestyle='--', color='k', alpha=0.3)